In [1]:
# default_exp test

In [21]:
# export 
import numpy as np
import torch
import torchvision
from time import time
from torchvision import datasets, transforms
from torch import nn, optim


In [22]:
# export
def get_test_resources(force_cpu=False):
    if force_cpu:
        device = torch.device("cpu")
    else:
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    transform = transforms.Compose([transforms.ToTensor(),
                                  transforms.Normalize((0.5,), (0.5,)),
                                  ])

    trainset = datasets.MNIST('resources/mnist/train', download=True, train=True, transform=transform)
    valset = datasets.MNIST('resources/mnist/test', download=True, train=False, transform=transform)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
    valloader = torch.utils.data.DataLoader(valset, batch_size=64, shuffle=True)

    dls = {
        'train':trainloader,
        'test':valloader,
        'valid':valloader,
    }

    input_size = 784
    hidden_sizes = [128, 64]
    output_size = 10
    
    class Loss(nn.Module):
        def __init__(self):
            super().__init__()
            self.loss_fn = nn.CrossEntropyLoss()
        def forward(self,x,y):
            loss = self.loss_fn(x,y)
            return loss,loss

    model = nn.Sequential(nn.Linear(input_size, hidden_sizes[0]),
                          nn.ReLU(),
                          nn.Linear(hidden_sizes[0], hidden_sizes[1]),
                          nn.ReLU(),
                          nn.Linear(hidden_sizes[1], output_size)
                         )
    model = model.to(device)
    optimizer = optim.SGD(model.parameters(), lr=0.003, momentum=0.9)
    loss_func = Loss()
    return model,dls,loss_func,optimizer,device


In [107]:
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted 01nabla2.ipynb.
Converted 02_learner.ipynb.
Converted 05_progressbar.ipynb.
Converted 99_tests.ipynb.
Converted index.ipynb.


---

In [108]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [109]:

from nabla2.nabla2 import Learner,accuracy
from nabla2.test import get_test_resources


In [110]:
model,dls,loss_func,optimizer,device = get_test_resources(True)

def foo(p,g):
    return -1.0


learner = Learner(model,dls,loss_func,optimizer,device=device,eval_sets={'valid':accuracy,'train':accuracy,'test':[accuracy],'twst1':accuracy})

def on_batch_begin(data_batch,phase='train'):
    xb,yb = data_batch
    xb = xb.view(-1, 784)
    return xb,yb
learner.on_batch_begin=on_batch_begin


>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
25/05/2020 11:21:52


In [111]:
learner.fit(2)

      accuracy                   error            loss                                                 
EPOCH     test  train  valid     twst1            test             train           valid
1        0.906  0.905  0.906  na in ds  [0.323, 0.323]  [[0.675, 0.675]]  [0.323, 0.323]
2        0.926  0.926  0.926  na in ds  [0.248, 0.248]  [[0.288, 0.288]]  [0.249, 0.249]               


In [112]:
learner.eval_metrics()

defaultdict(dict,
            {'valid': {'accuracy': array('0.926', dtype='<U5'),
              'loss': array(['0.251', '0.251'], dtype='<U5')},
             'train': {'accuracy': array('0.926', dtype='<U5'),
              'loss': array(['0.252', '0.252'], dtype='<U5')},
             'test': {'accuracy': array('0.926', dtype='<U5'),
              'loss': array(['0.247', '0.247'], dtype='<U5')},
             'twst1': {'error': 'na in ds'}})

In [114]:
# learner.recorder.get_loss_df()

In [115]:
LLlut = learner.recorder.lut

In [117]:
R = _recorder()
R.lut = LLlut.copy()

In [120]:
t,v = R.get_loss_df()

In [121]:
t

,epoch_num,iter_num,lr,loss_0,loss_1
0,0,0,0.003,2.301287,2.301287
1,0,1,0.003,2.300428,2.300428
2,0,2,0.003,2.278993,2.278993
3,0,3,0.003,2.290361,2.290361
4,0,4,0.003,2.298497,2.298497
...,...,...,...,...,...
1873,1,1872,0.003,0.257591,0.257591
1874,1,1873,0.003,0.241604,0.241604
1875,1,1874,0.003,0.380979,0.380979
1876,1,1875,0.003,0.175926,0.175926


In [124]:
v

,epoch_num,iter_num,lr
